### Code for Training timeleft Neural Network

In [5]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn import preprocessing as preproc

import os
import random

In [102]:
#Load Data
DATA_DIR = "DataSets"
file_name = "Time Remaining In Activity Data Set.xlsx"
file_name = os.path.join(DATA_DIR, file_name)

df = pd.read_excel(file_name)
activity_dic = {1: "Beast Mode",
               2: "Studying",
               3: "Beach",
               4: "Relaxing"}
df.head()

,Activity 1,Activity 2,Time Remaining(%),Energy Change
0,4,2,0.8733,1
1,1,3,0.5494,1
2,2,1,0.2009,1
3,2,4,0.3034,1
4,4,4,0.9102,0


In [111]:
# Preprocess Data
def preproc_df(df):
    ### ONE HOT ENCODING FOR ACTIVITYS
    act1_cols = ["act11", "act12", "act13", "act14"]
    act2_cols = ["act21", "act22", "act23", "act24"]
    
    actcols = act1_cols+act2_cols
    df_r, df_c = df.shape
    
    activity_dic = {}
    
    for col in actcols:
        activity_dic[col] = [0]*df_r
    
    act_df = pd.DataFrame(activity_dic, columns = actcols)
    
    for index, row in act_df.iterrows():
        act1_in = int(df["Activity 1"].iloc[index])
        act2_in = int(df["Activity 2"].iloc[index])        
        row["act1" + str(act1_in)] = 1 
        row["act2" + str(act2_in)] = 1 
        
    # Concatnate one hot encoding df to regular df
    
    df = pd.concat([df,act_df], axis=1)
    df.dropna(inplace=True)
    
    data = []
    for row in df.values:
        EnergyChange = row[3]
        activitys = row[-8:]
        timeleft = row[2]
        xdata = list(activitys)+[timeleft]
        data.append([np.array(xdata),EnergyChange])
        
    random.shuffle(data)
        
    X_data = []
    y_data = []
    
    for X,y in data:
        X_data.append(X)
        y_data.append(y)
    
        
    return np.array(X_data), np.array(y_data),df

In [113]:
X,y,ndf= preproc_df(df)
ndf.head()

,Activity 1,Activity 2,Time Remaining(%),Energy Change,act11,act12,act13,act14,act21,act22,act23,act24
0,4,2,0.8733,1,0,0,0,1,0,1,0,0
1,1,3,0.5494,1,1,0,0,0,0,0,1,0
2,2,1,0.2009,1,0,1,0,0,1,0,0,0
3,2,4,0.3034,1,0,1,0,0,0,0,0,1
4,4,4,0.9102,0,0,0,0,1,0,0,0,1


In [128]:
# Making Model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])


In [132]:
model.fit(X,y,epochs=12,validation_split=0.2)

Train on 3072 samples, validate on 768 samples
Epoch 1/12
3072/3072 [==============================] - 0s 42us/sample - loss: 0.2629 - accuracy: 0.8268 - val_loss: 0.3163 - val_accuracy: 0.7956
Epoch 2/12
3072/3072 [==============================] - 0s 46us/sample - loss: 0.2704 - accuracy: 0.8327 - val_loss: 0.3105 - val_accuracy: 0.8112
Epoch 3/12
3072/3072 [==============================] - 0s 50us/sample - loss: 0.2680 - accuracy: 0.8298 - val_loss: 0.3414 - val_accuracy: 0.7917
Epoch 4/12
3072/3072 [==============================] - 0s 50us/sample - loss: 0.2759 - accuracy: 0.8317 - val_loss: 0.3313 - val_accuracy: 0.8047
Epoch 5/12
3072/3072 [==============================] - 0s 53us/sample - loss: 0.2696 - accuracy: 0.8249 - val_loss: 0.3132 - val_accuracy: 0.8047
Epoch 6/12
3072/3072 [==============================] - 0s 54us/sample - loss: 0.2732 - accuracy: 0.8346 - val_loss: 0.3301 - val_accuracy: 0.8021
Epoch 7/12
3072/3072 [==============================] - 0s 55us/sample 